In [35]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display

from atj_algotrading.backtester import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [36]:
symbol = 'DE40'
start_dt = datetime.now() - timedelta(days= 365 * 10)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)

ohlc['month'] = ohlc['time'].dt.month

display(ohlc)

fig = create_price_fig(ohlc)
display(fig)

,time,open,high,low,close,month
0,2014-10-06,9275.90,9347.50,9143.30,9162.00,10
1,2014-10-07,9161.70,9178.30,8992.30,8994.40,10
2,2014-10-08,8991.40,9125.30,8959.80,9117.10,10
3,2014-10-09,9115.90,9141.50,8883.20,8894.30,10
4,2014-10-10,8894.90,8960.30,8764.30,8770.00,10
...,...,...,...,...,...,...
2536,2024-09-25,18992.64,18998.38,18841.69,18918.18,9
2537,2024-09-26,18923.81,19293.31,18913.65,19277.50,9
2538,2024-09-27,19280.08,19492.90,19227.40,19403.20,9
2539,2024-09-30,19374.89,19476.60,19305.90,19354.69,9


In [37]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 10000 / data['open']
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['month'] == 11 and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy')
    
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if data['month'] == 12:
            orders.close_trade(trade)

In [38]:
# backtest parameters
starting_balance = 10000
currency = 'EUR'
exchange_rate = 1
commission = -1

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,DE40,buy,1.072524,2014-11-03 00:00:00,9323.8,2014-12-01 00:00:00,9924.0,0,0,None,643.73,-1.072524,642.657476,642.657476,10642.657476
1,closed,DE40,buy,0.929368,2015-11-02 00:00:00,10760.0,2015-12-01 00:00:00,11411.0,0,0,None,605.02,-0.929368,604.090632,1246.748108,11246.748108
2,closed,DE40,buy,0.934324,2016-11-01 00:00:00,10702.92,2016-12-01 00:00:00,10618.95,0,0,None,-78.46,-0.934324,-79.394324,1167.353783,11167.353783
3,closed,DE40,buy,0.750243,2017-11-01 00:00:00,13329.01,2017-12-01 00:00:00,13027.5,0,0,None,-226.21,-0.750243,-226.960243,940.39354,10940.39354
4,closed,DE40,buy,0.875208,2018-11-01 00:00:00,11425.85,2018-12-03 00:00:00,11457.91,0,0,None,28.06,-0.875208,27.184792,967.578332,10967.578332
5,closed,DE40,buy,0.775452,2019-11-01 00:00:00,12895.7,2019-12-02 00:00:00,13240.47,0,0,None,267.35,-0.775452,266.574548,1234.152879,11234.152879
6,closed,DE40,buy,0.86273,2020-11-02 00:00:00,11591.11,2020-12-01 00:00:00,13309.34,0,0,None,1482.37,-0.86273,1481.50727,2715.660149,12715.660149
7,closed,DE40,buy,0.635074,2021-11-01 00:00:00,15746.2,2021-12-01 00:00:00,15216.7,0,0,None,-336.27,-0.635074,-336.905074,2378.755075,12378.755075
8,closed,DE40,buy,0.75449,2022-11-01 00:00:00,13253.98,2022-12-01 00:00:00,14589.43,0,0,None,1007.58,-0.75449,1006.82551,3385.580585,13385.580585
9,closed,DE40,buy,0.673488,2023-11-01 00:00:00,14848.08,2023-12-01 00:00:00,16234.88,0,0,None,933.99,-0.673488,933.316512,4318.897097,14318.897097


In [39]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [40]:
backtest_fig = bt.visualize_backtest(indicators=['sma_20'])
backtest_fig

ValueError: All arguments should have the same length. The length of argument `y` is 1, whereas the length of  previously-processed arguments ['time'] is 2541

In [ ]:
evaluate_backtest(bt.trades)